Data Science 874
**bold text**
Department of Industrial Engineering &

School for Data Science and Computational Thinking

Stellenbosch University

By Nomampondomise Ngxongxela

Kabelo Theoha

Sydney Nyombi

Angel Musonda

In [296]:
#MOUNT Google Drive
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

#Load the dataset into a Pandas DataFrame
Bank_Raw=pd.read_csv('/content/drive/My Drive/Datasets/Bank_Customer_Churn_Prediction.csv', sep=',')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**1.** **View the Data (Data Understanding)**

The dataset is available in the form of a single file. The name had to be changed by replacing the spaces with underscores tobe readable. This step is to view the dataset and determine the required preparation steps required before the data can used to model. The shape (number of rows and columns) will reveal the size of the data. A print out of the first ten rows will reveal the features and type of data available. A print out of the data information will reveal the presence of null values on each feature as well as the data type. The above information will enable an assessment to be made on whether the imputing of missing values, encoding of features, balancing and or scaling is required before the dataset can be used to model. 

**1.1. Data Shape**

In [297]:
# View the size of the dataset
Bank_Raw.shape

(10000, 12)

**1.2. Data Heads (First 10 rows)**

In [298]:
#View Data sample
Bank_Raw.head(10)

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,15574012,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,15592531,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,15656148,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,15792365,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,15592389,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


**1.3. Data Information**

In [299]:
#Dataset Information
Bank_Raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


**Findings of Data Viewing:**


*   The target feature is churn indicting customers who have left. This is a classification prediction problem.
*   There are no null values
There are instances where the 'active_member' and 'churn' features both 

*   There are instances where the 'active_member' and 'churn' features both show '1' which means a customer is both an active member and has been churned (have left). This needs to be corrected.
*   Given the varying scales of the features, scaling is recommended.


*   The 'Country' and 'Gender' features are categorical and will need to be converted to numerical features



*   Further analysis required to view skewness, balance, correlation etc.

**1.4. Exploratory Data Analysis on the Features**

In [300]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
import pandas_profiling
from pandas_profiling import ProfileReport

#EDA using pandas-profiling
profile = ProfileReport(Bank_Raw, explorative=True)
profile.to_notebook_iframe()

Output hidden; open in https://colab.research.google.com to view.

**2. Data Preparing (Engineering the Data)**


* 2.1.Replace categorical data with numerical data
* 2.2. Scale input selected features 
* 2.3.Conduct Information Gain Analysis to determine best features to fit model
* 2.4.Pre-process data by splitting into Training, valaidation and test data.

**2.1. Convert Categorical Features to Numerical** 

In [301]:
#replace Spain, Germany & France in  the 'Country' feature
#replace Male & Female in  the 'Gender' feature
Bank_Raw[['country']] = Bank_Raw[['country']].replace(to_replace={'Spain':3, 'Germany':2,'France':1})
Bank_Raw[['gender']] = Bank_Raw[['gender']].replace(to_replace={'Male':0, 'Female':1})
Bank_Raw.head(5)

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,1,1,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,3,1,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,1,1,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,1,1,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,3,1,43,2,125510.82,1,1,1,79084.10,0


**2.3. Scale Selected Input features**

In [302]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()  # betwen 0 and 1 

# I am keeping the "customer_id", "age" feature from being scaled. I am also not scaling the columns with 1 and 0
Bank_Raw[['credit_score','tenure','balance','products_number','estimated_salary']] = min_max_scaler.fit_transform(Bank_Raw[['credit_score','tenure','balance','products_number','estimated_salary']])

#Check whether the values have been scaled
Bank_Raw.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,0.538,1,1,42,0.2,0.000000,0.000000,1,1,0.506735,1
1,15647311,0.516,3,1,41,0.1,0.334031,0.000000,0,1,0.562709,0
2,15619304,0.304,1,1,42,0.8,0.636357,0.666667,1,0,0.569654,1
3,15701354,0.698,1,1,39,0.1,0.000000,0.333333,0,0,0.469120,0
4,15737888,1.000,3,1,43,0.2,0.500246,0.000000,1,1,0.395400,0


**2.3. Conduct Information Gain Analysis**

In [303]:
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html
from sklearn.feature_selection import mutual_info_classif

#this is what you have to split into training and test data
X_g = Bank_Raw.drop(['churn',"customer_id"], axis=1)
y_g = Bank_Raw["churn"]

information_gain_results = dict(zip(list(X.columns), mutual_info_classif(X_g, y_g, discrete_features=True)))

#Sort the values in the info gain dictionary
information_gain_results = sorted(information_gain_results.items(), key=lambda x: x[1], reverse=True)

#Display the information gain (feature_name, ig_score)
#The higher the score, the most usefull the feature
#You can select your own cut off point.
information_gain_results

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.1

[('active_member', 0.3600380503175834),
 ('balance', 0.07431613249395502),
 ('estimated_salary', 0.06995349771085277),
 ('credit_score', 0.02671875330009316),
 ('country', 0.014017045772466472),
 ('age', 0.005652197849223051),
 ('products_number', 0.000702813271552756)]

**2.4. Pre-Process Data (Training, Validation, Test)**

In [304]:
from sklearn.model_selection import train_test_split
#Splitiing data into 60% (6000) traning , 20% (2000) validation and 20% (2000) testing dataset. Input has the 'optimum features'

#Identify input and output
X= Bank_Raw.drop(['churn',"customer_id","credit_card","tenure","gender"], axis=1)
y= Bank_Raw["churn"]

#First split the trainingV/test (80/20) 
X_trainV, X_test, y_trainV, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

#Secondly split the trainingV set into traning and validation (60/20), 25% of the trainingV 80% from above 
X_train, X_val, y_train, y_val = train_test_split(X_trainV, y_trainV, test_size=0.25, random_state=33)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print("The data is ready for modelling.")

(6000, 7)
(2000, 7)
(2000, 7)
The data is ready for modelling.


**4. Modelling**

In this Step, Four Algorithms will be used and evaluted using the validation data. The best(most accurate) algorithm will be selected and then the test data will be used to evaluate the selected algorithm. In the Evalaution, metrics such as accuracy, recall, F1 score and precision will be used to determine the best model to use.

The model seeks to predict whether a bank customer will leave the bank or not given multiple features such as age, bank balance, estimated salary etc. This is therefore a classification problem.As this is a clasiffication problem,the following algorithms will be tested:
1. Decision Trees Classifier
2. K Nearest Neibours (KNN) Classifier
3. Random Forest Classifier
4. Feed Forward Neural Network(FNN) Classifier

**4.1. Implement a Decision Tree Classifier**

In [305]:
from sklearn.tree import DecisionTreeClassifier

#instantiate and fit() the model
DR_model = DecisionTreeClassifier(random_state =43)
Deci_Class=DR_model.fit(X_train, y_train)

In [306]:
#get som predictions
y_val_pred1 = Deci_Class.predict(X_val)

#Print a couple of predictions
print(list(y_val_pred1)[0:5])

[0, 1, 0, 0, 1]


In [307]:
#Metrics for the model evaluated using Validation Data

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
import numpy as np

Accuracy1=accuracy_score(y_val,y_val_pred1)
Recall1=recall_score(y_val,y_val_pred1,average='weighted')
f1_score1=f1_score(y_val,y_val_pred1,average='weighted')
Precision1=precision_score(y_val,y_val_pred1,average='weighted')


print("Accuracy1:"+str(round(100*Accuracy1,3)))
print("Recall1:"+str(round(100*Recall1,3)))
print("F1-Score1:"+str(round(100*f1_score1,3)))
print("Precision1:"+str(round(100*Precision1,3)))

Accuracy1:80.05
Recall1:80.05
F1-Score1:80.336
Precision1:80.67


**4.2. Implement a KNN Classifier**

In [308]:
from sklearn.neighbors import KNeighborsClassifier

#Build the model [fit(), you do some predictions]
neigh=KNeighborsClassifier(n_neighbors=17)
K_neigh=neigh.fit(X_train, y_train)

In [309]:
#Predictions
y_val_pred2 = K_neigh.predict(X_val)
y_val_pred2

array([0, 1, 0, ..., 0, 0, 0])

In [310]:
#Metrics for the model evaluated using Validation Data
Accuracy2=accuracy_score(y_val,y_val_pred2)
Recall2=recall_score(y_val,y_val_pred2,average='weighted')
f1_score2=f1_score(y_val,y_val_pred2,average='weighted')
Precision2=precision_score(y_val,y_val_pred2,average='weighted')


print("Accuracy2:"+str(round(100*Accuracy2,3)))
print("Recall2:"+str(round(100*Recall2,3)))
print("F1-Score2:"+str(round(100*f1_score2,3)))
print("Precision2:"+str(round(100*Precision2,3)))

Accuracy2:83.85
Recall2:83.85
F1-Score2:81.442
Precision2:82.392


**4.3. Implement a Random Forest Classifier**

In [311]:
from sklearn.ensemble import RandomForestClassifier

#instantiate and fit() the model
RC_model=RandomForestClassifier(n_estimators=1000)
Rand_ClassF=RC_model.fit(X_train, y_train)

In [312]:
#Predictions
y_val_pred3 = Rand_ClassF.predict(X_val)
y_val_pred3

array([0, 1, 0, ..., 0, 0, 0])

In [313]:
#Metrics for the model evaluated using Validation Data
Accuracy3=accuracy_score(y_val,y_val_pred3)
Recall3=recall_score(y_val,y_val_pred3,average='weighted')
f1_score3=f1_score(y_val,y_val_pred3,average='weighted')
Precision3=precision_score(y_val,y_val_pred3,average='weighted')


print("Accuracy3:"+str(round(100*Accuracy3,3)))
print("Recall3:"+str(round(100*Recall3,3)))
print("F1-Score3:"+str(round(100*f1_score3,3)))
print("Precision3:"+str(round(100*Precision3,3)))

Accuracy3:87.05
Recall3:87.05
F1-Score3:85.94
Precision3:86.294


**4.4. Implement a FNN =MLP Classifier**

In [314]:
#Feed Foward Neural Network(FNN) = MLP(Multi Layer Perception)

from sklearn.neural_network import MLPClassifier

MLP_Model=MLPClassifier(random_state=55,hidden_layer_sizes=(500),activation='relu',max_iter=1000).fit(X_train,y_train)

In [315]:
#Predictions
y_val_pred4 = MLP_Model.predict(X_val)
y_val_pred4

array([0, 1, 0, ..., 0, 0, 0])

In [316]:
#Metrics for the model evaluated using Validation Data
Accuracy4=accuracy_score(y_val,y_val_pred4)
Recall14=recall_score(y_val,y_val_pred4,average='weighted')
f1_score4=f1_score(y_val,y_val_pred4,average='weighted')
Precision4=precision_score(y_val,y_val_pred4,average='weighted')

print("Accuracy4:"+str(round(100*Accuracy4,3)))
print("Recall4:"+str(round(100*Recall4,3)))
print("F1-Score4:"+str(round(100*f1_score4,3)))
print("Precision4:"+str(round(100*Precision4,3)))

Accuracy4:87.25
Recall4:52.099
F1-Score4:85.758
Precision4:86.866


**5. Evaluate Model Using Test Data (Random Forest Classifier)**

In [317]:
#Predictions
y_pred = Rand_ClassF.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [318]:
#Metrics for the model evaluated using Validation Data
Accuracy=accuracy_score(y_test,y_pred)
Recall=recall_score(y_test,y_pred,average='weighted')
f1_score=f1_score(y_test,y_pred,average='weighted')
Precision=precision_score(y_test,y_pred,average='weighted')


print("Accuracy:",round(100*Accuracy,3))
print("Recall:",round(100*Recall,3))
print("F1-Score:",round(100*f1_score,3))
print("Precision:",round(100*Precision,3))

Accuracy: 85.25
Recall: 85.25
F1-Score: 83.914
Precision: 84.03


**5. Exporting for Visualisation**